### Training Configuration and Environment Setup

This section establishes the foundation for the deep learning experiment. It imports necessary libraries, defines the experimental scope (dataset and features), sets up the file system for results, and initializes critical hyperparameters.

#### 1. Library Imports
The script leverages **TensorFlow/Keras** for building the deep learning architecture. Notably, it imports a diverse set of layers—`Conv2D` (CNN), `LSTM` (RNN), and `MultiHeadAttention`—confirming the **hybrid** nature of the model.
* **Evaluation:** `sklearn` metrics are imported to calculate Precision, Recall, and F1-score.
* **Explainability:** `shap` is imported for feature importance analysis.
* **Visualization:** `matplotlib` is set up for plotting training history and interpretation heatmaps.

#### 2. Experiment Configuration
The script uses constant flags to control the specific experimental setup. By modifying variables like `DATASET` (e.g., `MPOWER_DATASET`) or `MODE`, the user can switch between different training scenarios without altering the core logic.

#### 3. Dynamic Path Management
To ensure a structured workflow, the code dynamically constructs file paths based on the selected configuration:
* **Input Path:** Points to the `.npz` feature file generated by the preprocessing pipeline.
* **Output Paths:** Automatically creates directories to separate model checkpoints (`best_model.keras`), evaluation logs, SHAP analysis values, and Grad-CAM heatmaps. This ensures that results from different experiments do not overwrite each other.

#### 4. Hyperparameters & Optimization
Key training parameters are defined here to control the model's learning process:
* **Epochs (30) & Batch Size (32):** Standard settings for convergence on this dataset size.
* **Learning Rate (0.001):** A standard starting point for the Adam optimizer.
* **Regularization:**
    * **Dropout (0.5):** A high dropout rate is used to aggressively prevent overfitting, which is critical for medical datasets with limited samples.
    * **L2 Regularization (0.01):** Applied to penalize large weights and further improve generalization.

#### 5. Model Checkpointing
A `ModelCheckpoint` callback is initialized to monitor the **Validation AUC** (`val_auc`).
* **Strategy:** It saves the model weights *only* when the AUC improves.
* **Rationale:** In medical diagnostics, where class imbalance is common, maximizing AUC is often more clinically relevant than maximizing simple accuracy.

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dropout, Flatten,
                                     Dense, LSTM, MultiHeadAttention, Concatenate, Reshape)
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from tqdm import tqdm
from scipy.stats import ttest_ind
import pandas as pd
import shap
from tensorflow.keras.saving import register_keras_serializable
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage.transform import resize
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm

# =============================================================================
# --- Configuration ---
# =============================================================================

# --- 1. Core Paths ---
ITALIAN_DATASET = "ITALIAN_DATASET"
UAMS_DATASET = "UAMS_DATASET"
MPOWER_DATASET = "MPOWER_DATASET"

MODE_ALL_VALIDS = "ALL_VALIDS"
MODE_A = "A"

FEATURE_MODE_DEFAULT = "DEFAULT"

MODEL_NAME = "parallel"

# /////////// SELCET HERE \\\\\\\\\\\
# ----------------------------------
DATASET = MPOWER_DATASET
MODE = MODE_A
FEATURE_MODE = FEATURE_MODE_DEFAULT
# ----------------------------------

In [ ]:
dataset = ""
elif DATASET == UAMS_DATASET:
    dataset = "UAMS"
elif DATASET == MPOWER_DATASET:
    dataset = "mPower"
elif DATASET == ITALIAN_DATASET:
    dataset = "Italian"

# Path Setup
FEATURES_FILE_PATH = os.path.join(os.getcwd(), dataset, "data", f"features_{MODE}_{FEATURE_MODE}.npz")

MODEL_PATH = os.path.join(os.getcwd(), dataset, f"results_{MODE}_{FEATURE_MODE}", MODEL_NAME)
os.makedirs(MODEL_PATH, exist_ok=True)
EVALUATION_FILE_PATH = os.path.join(MODEL_PATH, "evaluation.csv")
HISTORY_SAVE_PATH = os.path.join(MODEL_PATH, "history.csv")
BEST_MODEL_PATH = os.path.join(MODEL_PATH, "best_model.keras")

SHAP_OUTPUT_PATH = os.path.join(MODEL_PATH, "shap_analysis")
GRADCAM_OUTPUT_PATH = os.path.join(MODEL_PATH, "gradcam_analysis")
os.makedirs(SHAP_OUTPUT_PATH, exist_ok=True)
os.makedirs(GRADCAM_OUTPUT_PATH, exist_ok=True)

# Hyperparameters
EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
L2_STRENGTH = 0.01

# Model Checkpoint Callback
checkpoint_cb = ModelCheckpoint(BEST_MODEL_PATH, monitor='val_auc', mode='max', save_best_only=True, verbose=1)

In [ ]:
def load_data(feature_file_path):

    print(f"--- Loading data from {feature_file_path} ---")
    with np.load(feature_file_path) as data:
        labels = data['labels']
        mel_spectrogram = data['mel_spectrogram']
        mfcc = data['mfcc']
        X = np.concatenate((mel_spectrogram, mfcc), axis=1)
        return X, labels

### Model Architecture: ParkinsonDetectorModel

The `ParkinsonDetectorModel` is a custom Keras model designed for the binary classification of Parkinson's Disease using spectral audio features. It employs a **hybrid architecture** that sequentially extracts spatial features using Convolutional Neural Networks (CNNs) and then processes these features in parallel using Recurrent Neural Networks (LSTMs) and Multi-Head Attention mechanisms.

#### 1. Class Structure and Initialization
The class inherits from `tensorflow.keras.models.Model` and is decorated with `@register_keras_serializable()`, ensuring the custom model structure can be saved and loaded correctly.

**Key Components in `__init__`:**
* **Input Reshaping:** A `Reshape` layer adds a channel dimension (depth=1) to the 2D input (Mel Spectrograms + MFCCs) to make it compatible with 2D Convolution.
* **CNN Blocks:**
    * Two consecutive blocks of `Conv2D` layers (64 filters, 5x5 kernel) followed by `MaxPooling2D` and `Dropout`.
    * These layers are responsible for learning spatial hierarchies and local patterns in the time-frequency domain.
    * L2 regularization (`kernel_regularizer=l2`) is applied to all convolutional layers to prevent overfitting.
* **Parallel Branches:**
    * **Attention:** A `MultiHeadAttention` layer (2 heads) to identify and weigh critical segments of the feature sequence.
    * **LSTM:** A dual-layer `LSTM` stack (128 units) to capture temporal dependencies and long-term evolution of the signal.
* **Fusion & Output:**
    * `Concatenate` layer to merge the outputs from the CNN, Attention, and LSTM streams.
    * A Dense `bottleneck` layer (128 units) for feature compression.
    * A final `Dense` output layer with a **Sigmoid** activation for binary probability (Healthy vs. Parkinson's).

#### 2. Forward Pass Logic (`call` method)
The `call` method defines the data flow through the network:

1.  **Feature Extraction (Sequential):**
    * The input passes through the two CNN blocks.
    * The output `x` (after the second pooling) represents the high-level spatial features.

2.  **Branching:**
    * **Branch 1 (Spatial):** The CNN output `x` is flattened (`cnn_flat`) to preserve spatial feature information.
    * **Sequence Preparation:** The CNN output `x` is reshaped into a sequence format `(batch, time_steps, features)` to serve as input for the temporal branches.

3.  **Parallel Processing:**
    * **Branch 2 (Attention):** The prepared sequence is passed through Self-Attention (`query=key=value=sequence`). The result highlights important time steps.
    * **Branch 3 (Temporal):** The same sequence passes through the LSTM layers to model temporal dynamics.

4.  **Fusion and Classification:**
    * The outputs of all three branches (`cnn_flat`, `att_flat`, `lstm_out`) are concatenated into a single rich feature vector.
    * This vector passes through the bottleneck layer and finally the output layer to produce the prediction.

#### 3. Serialization Support
The model includes `get_config` and `from_config` methods. These are essential for saving the model's architecture and custom hyperparameters (like `input_shape`) to disk and reconstructing it later without recompiling the code.

#### 4. Builder Function
The `build_model(input_shape)` helper function wraps this custom class instantiation within the Keras Functional API paradigm. This is standard practice to ensure the model is built and the input shape is correctly inferred before training begins.

In [ ]:
# =============================================================================
# --- Model Architecture ---
# =============================================================================

@register_keras_serializable()
class ParkinsonDetectorModel(Model):
    def __init__(self, input_shape, **kwargs):
        super(ParkinsonDetectorModel, self).__init__(**kwargs)
        self.input_shape_config = input_shape

        self.reshape_in = Reshape((input_shape[0], input_shape[1], 1))
        self.conv1a = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same')
        self.conv1b = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same')
        self.pool1 = MaxPooling2D(5)
        self.drop1 = Dropout(DROPOUT_RATE)
        self.conv2a = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same')
        self.conv2b = Conv2D(64, 5, activation='relu', kernel_regularizer=l2(L2_STRENGTH), padding='same', name='last_conv_layer')
        self.pool2 = MaxPooling2D(5, name='cnn_output')
        self.drop2 = Dropout(DROPOUT_RATE)
        self.flatten_cnn = Flatten()
        self.attention = MultiHeadAttention(num_heads=2, key_dim=64, name='attention_output')
        self.flatten_att = Flatten()
        self.lstm1 = LSTM(128, return_sequences=True)
        self.lstm2 = LSTM(128, return_sequences=False, name='lstm_output')
        self.drop_lstm = Dropout(DROPOUT_RATE)
        self.concat = Concatenate()
        self.dense_bottleneck = Dense(128, activation='relu', name='bottleneck_features')
        self.dense_output = Dense(1, activation='sigmoid')


    def call(self, inputs, training=False):
        x = self.reshape_in(inputs)
        x = self.conv1a(x)
        x = self.conv1b(x)
        x = self.pool1(x)
        x = self.drop1(x, training=training)
        x = self.conv2a(x)
        x = self.conv2b(x)
        cnn_branch_output = self.pool2(x)
        x = self.drop2(cnn_branch_output, training=training)

        cnn_flat = self.flatten_cnn(x)

        shape = tf.shape(x)
        sequence = tf.reshape(x, [-1, shape[1] * shape[2], shape[3]])

        att_branch_output = self.attention(query=sequence, key=sequence, value=sequence)
        att_flat = self.flatten_att(att_branch_output)

        lstm_seq = self.lstm1(sequence)
        lstm_branch_output = self.lstm2(lstm_seq)
        lstm_out = self.drop_lstm(lstm_branch_output, training=training)

        concatenated = self.concat([cnn_flat, att_flat, lstm_out])
        bottleneck = self.dense_bottleneck(concatenated)
        final_output = self.dense_output(bottleneck)

        return final_output

    def get_config(self):
        config = super(ParkinsonDetectorModel, self).get_config()
        config.update({"input_shape": self.input_shape_config})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

def build_model(input_shape: tuple) -> Model:
    """Builds the hybrid model by wrapping the custom class in a Functional API model."""
    print("--- Building the model ---")
    inputs = Input(shape=input_shape)
    parkinson_detector = ParkinsonDetectorModel(input_shape=input_shape)
    outputs = parkinson_detector(inputs)
    model = Model(inputs=inputs, outputs=outputs)
    print("Model built successfully.")
    return model

In [ ]:
# =============================================================================
# --- Model Performance ---
# =============================================================================
def save_metrics_to_csv(y_true, y_pred_proba, filename="classification_report.csv", threshold=0.5):
    y_true = np.array(y_true)
    y_pred_binary = (np.array(y_pred_proba) > threshold).astype(int)

    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    total_samples = cm.sum()
    if total_samples == 0:
        tn_percent, fp_percent, fn_percent, tp_percent = 0, 0, 0, 0
    else:
        tn_percent = (tn / total_samples) * 100
        fp_percent = (fp / total_samples) * 100
        fn_percent = (fn / total_samples) * 100
        tp_percent = (tp / total_samples) * 100

    precision = precision_score(y_true, y_pred_binary, zero_division=0)
    recall = recall_score(y_true, y_pred_binary, zero_division=0)
    f1 = f1_score(y_true, y_pred_binary, zero_division=0)
    sensitivity = recall

    report_data = {
        'Metric': [
            'True Positive (TP)',
            'True Negative (TN)',
            'False Positive (FP)',
            'False Negative (FN)',
            'Precision',
            'Recall (Sensitivity)',
            'F1-Score'
        ],
        'Value': [
            f"{tp} ({tp_percent:.2f}%)",
            f"{tn} ({tn_percent:.2f}%)",
            f"{fp} ({fp_percent:.2f}%)",
            f"{fn} ({fn_percent:.2f}%)",
            f"{precision:.4f}",
            f"{recall:.4f}",
            f"{f1:.4f}"
        ]
    }
    df = pd.DataFrame(report_data)

    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"The evaluation results is stored: {filename}")
    except Exception as e:
        print(f"Erorr while saving the evaluation report: {e}")


### Feature Extraction for Dimensionality Reduction Analysis

This module is designed to extract high-dimensional feature vectors from the internal layers of the trained model. Specifically, it targets the point where features from the CNN, LSTM, and Attention branches are concatenated, just before they enter the final dense classification layers. These extracted vectors are essential for visualizing the model's learned representation space using techniques like PCA, t-SNE, or LDA.

#### Class: `DenseLayerInputExtractor`

This class handles the creation of a sub-model that isolates the intermediate output of the network.

**1. Initialization (`__init__`)**
The constructor initializes the class and immediately calls `_setup_extraction_model` to prepare the internal Keras model for feature extraction.

**2. Model Setup (`_setup_extraction_model`)**
This is the core logic that reconstructs the forward pass of the model up to the concatenation layer.
* **Layer Identification:** It iterates through the trained model's layers to find the specific `ParkinsonDetectorModel` instance.
* **Graph Reconstruction:** It re-defines the flow of tensors through the specific branches:
    * **CNN Branch:** Passes input through convolutional and pooling layers to extract spatial features (`cnn_flat`).
    * **Sequence Reshaping:** Reshapes the CNN output to be compatible with sequence processing.
    * **Attention Branch:** Computes self-attention on the sequence (`att_flat`).
    * **LSTM Branch:** Processes the sequence through LSTM layers to capture temporal dynamics (`lstm_out`).
* **Concatenation:** Merges the outputs (`cnn_flat`, `att_flat`, `lstm_out`) into a single vector.
* **Model Creation:** Instantiates a new Keras `Model` (`self.extraction_model`) where the input matches the original model, but the output is this concatenated feature vector.

**3. Feature Extraction (`extract_and_save_features`)**
This method executes the extraction on a dataset and saves the results.
* **Sampling:** Allows extracting features from a subset of the data (e.g., 200 random samples) to reduce computational load for visualization, or processing the entire dataset. It handles random seeding for reproducibility.
* **Batch Processing:** Iterates through the data in batches to manage memory usage efficiently.
* **Output:** Compiles the extracted feature vectors and their corresponding labels into a dictionary and saves them as a compressed NumPy file (`.npz`). This file contains:
    * `concatenated_features`: The high-dimensional feature vectors.
    * `labels`: The ground truth class labels.
    * `sample_indices`: Indices of the samples used.

#### Helper Function: `extract_dense_layer_input_features`

This standalone function serves as a simplified entry point to the class.
* **Workflow:** It instantiates the `DenseLayerInputExtractor`.
* **Execution:** It calls `extract_and_save_features` with the test data and specifies the output filename (`dense_layer_input_features.npz`).
* **Purpose:** It encapsulates the extraction logic into a single function call for use in the main execution pipeline.

In [ ]:
class DenseLayerInputExtractor:
    """Extracts features that go into the dense layer for dimensionality reduction visualization"""

    def __init__(self, trained_model):
        self.parkinson_detector = None
        self.extraction_model = None
        self._setup_extraction_model(trained_model)

    def _setup_extraction_model(self, trained_model):
        """Setup model to extract concatenated features (dense layer input)"""
        print("Setting up dense layer input extraction model...")

        # Find the ParkinsonDetectorModel layer
        for layer in trained_model.layers:
            if isinstance(layer, ParkinsonDetectorModel):
                self.parkinson_detector = layer
                break

        if self.parkinson_detector is None:
            raise ValueError("ParkinsonDetectorModel not found in the trained model")

        def extract_dense_input_features(inputs):
            """Extract the concatenated features that go into the dense layer"""
            # Pass through CNN branch
            x = self.parkinson_detector.reshape_in(inputs)
            x = self.parkinson_detector.conv1a(x)
            x = self.parkinson_detector.conv1b(x)
            x = self.parkinson_detector.pool1(x)
            x_dropped = self.parkinson_detector.drop1(x, training=False)
            x = self.parkinson_detector.conv2a(x_dropped)
            x = self.parkinson_detector.conv2b(x)
            cnn_branch_output = self.parkinson_detector.pool2(x)
            x_dropped = self.parkinson_detector.drop2(cnn_branch_output, training=False)

            # Prepare sequence for attention and LSTM
            def reshape_for_sequence(x):
                shape = tf.shape(x)
                return tf.reshape(x, [-1, shape[1] * shape[2], shape[3]])

            sequence = Lambda(reshape_for_sequence, name='sequence_reshape')(x_dropped)

            # Attention branch
            att_branch_output = self.parkinson_detector.attention(
                query=sequence, key=sequence, value=sequence
            )

            # LSTM branch
            lstm_seq = self.parkinson_detector.lstm1(sequence)
            lstm_branch_output = self.parkinson_detector.lstm2(lstm_seq)
            lstm_out = self.parkinson_detector.drop_lstm(lstm_branch_output, training=False)

            # Flatten for concatenation
            cnn_flat = self.parkinson_detector.flatten_cnn(x_dropped)
            att_flat = self.parkinson_detector.flatten_att(att_branch_output)

            # This is what you need - the concatenated features (dense layer input)
            concatenated_features = self.parkinson_detector.concat([cnn_flat, att_flat, lstm_out])

            return concatenated_features

        # Create the extraction model
        inputs = Input(shape=trained_model.input_shape[1:])
        dense_input_features = extract_dense_input_features(inputs)
        self.extraction_model = Model(inputs=inputs, outputs=dense_input_features)

        print("✅ Dense layer input extraction model ready")

    def extract_and_save_features(self, X_data, y_data, output_path, sample_indices=None, batch_size=32):
        """Extract and save concatenated features (dense layer input)"""

        print(f"Input data shape: {X_data.shape}")

        # Handle sample selection
        if sample_indices is None:
            sample_indices = np.arange(len(X_data))
            print("Using all available samples")
        elif isinstance(sample_indices, int):
            available_samples = len(X_data)
            requested_samples = sample_indices

            if requested_samples > available_samples:
                print(f"⚠️  Warning: Requested {requested_samples} samples, but only {available_samples} available.")
                print(f"   Using all {available_samples} samples instead.")
                sample_indices = np.arange(available_samples)
            else:
                np.random.seed(42)
                sample_indices = np.random.choice(available_samples, requested_samples, replace=False)
                print(f"Randomly selected {requested_samples} samples from {available_samples} available.")

        print(f"Processing {len(sample_indices)} samples...")

        # Extract concatenated features in batches
        all_concatenated_features = []

        for i in tqdm(range(0, len(sample_indices), batch_size), desc="Extracting dense layer input features"):
            batch_indices = sample_indices[i:i+batch_size]
            batch_X = X_data[batch_indices]

            # Extract concatenated features for this batch
            batch_features = self.extraction_model.predict(batch_X, verbose=0)
            all_concatenated_features.append(batch_features)

        # Concatenate all batches
        concatenated_features = np.concatenate(all_concatenated_features, axis=0)
        print(f"Concatenated features shape: {concatenated_features.shape}")

        # Prepare data for your visualization script
        features_dict = {
            'concatenated_features': concatenated_features,  # This is what your viz script needs
            'labels': y_data[sample_indices],
            'sample_indices': sample_indices
        }

        # Save features
        np.savez_compressed(output_path, **features_dict)
        print(f"✅ Dense layer input features saved to: {output_path}")

        return output_path

def extract_dense_layer_input_features(trained_model, X_test, y_test, output_dir, num_samples=200):
    """Extract concatenated features that serve as input to the dense layer"""

    print("🔬 EXTRACTING DENSE LAYER INPUT FEATURES...")
    print("=" * 60)

    # Create extractor
    extractor = DenseLayerInputExtractor(trained_model)

    # Extract and save features
    output_path = os.path.join(output_dir, "dense_layer_input_features.npz")
    feature_file = extractor.extract_and_save_features(
        X_data=X_test,
        y_data=y_test,
        output_path=output_path,
        sample_indices=num_samples
    )

    print("=" * 60)
    print("✅ DENSE LAYER INPUT FEATURE EXTRACTION COMPLETED!")

    return feature_file


### Comprehensive Visualization and Analysis Tools

This section introduces advanced visualization functions designed to produce high-quality, publication-ready figures for analyzing model performance and feature separability. These functions are optimized for creating clear, informative plots suitable for posters and presentations.

#### 1. Feature Separability Analysis (`analyze_feature_separability`)
This function visualizes how well the model's learned features separate the two classes (Healthy vs. Parkinson's) using three dimensionality reduction techniques. It is optimized for large displays (XL/Poster version).

* **Dimensionality Reduction Techniques:**
    * **PCA (Principal Component Analysis):** Projects features onto the two principal components that capture the most variance.
    * **LDA (Linear Discriminant Analysis):** Projects features onto a single axis that maximizes class separation. The results are plotted as **density curves (KDE)** to show the distribution overlap.
    * **t-SNE (t-Distributed Stochastic Neighbor Embedding):** Visualizes high-dimensional clusters in 2D. A subset of samples (up to 1000) is used for efficiency.

* **Quantitative Metric (d-prime):**
    * Calculates the **d-prime** sensitivity index based on the LDA projection.
    * **Interpretation:** A higher d-prime indicates better separability between the healthy and pathological distributions. This value is annotated directly on the LDA plot.

* **Visualization Details:**
    * Generates a side-by-side subplot of PCA, LDA, and t-SNE.
    * Uses distinct colors (Blue for Healthy, Red for Parkinson's) and large fonts for readability.

#### 2. Side-by-Side Confusion Matrix & ROC (`plot_cm_and_roc_side_by_side`)
This function creates a unified figure combining the two most critical binary classification metrics.

* **Confusion Matrix (Left Panel):**
    * Displays raw counts and percentages for True Negatives, False Positives, False Negatives, and True Positives.
    * Annotates the axes with calculated **Specificity** and **Sensitivity (Recall)** scores.
    * Uses a heatmap with large annotations for clear visibility.

* **ROC Curve (Right Panel):**
    * Plots the Receiver Operating Characteristic curve.
    * Displays the **AUC (Area Under the Curve)** score in the legend.
    * Includes a diagonal reference line (random guess) for comparison.

#### 3. Full 3-Panel Evaluation (`plot_full_evaluation_square`)
This comprehensive function generates a single figure containing three key evaluation plots, maintaining a square aspect ratio for better visual balance.

* **Panel 1: Confusion Matrix:** Same enhanced heatmap as above.
* **Panel 2: ROC Curve:** Standard ROC plot with AUC.
* **Panel 3: Training History:**
    * Plots **Loss** (Training vs. Validation) on the primary y-axis (red).
    * Plots **AUC** (Training vs. Validation) on the secondary y-axis (blue).
    * This allows for simultaneous monitoring of convergence and overfitting (e.g., if validation loss starts rising while training loss falls).

All functions automatically save the generated high-resolution (300 DPI) images to the specified output directory.

In [ ]:
# =============================================================================
# --- New Imports for Enhanced Analysis and Plotting ---
# =============================================================================
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import os

# =============================================================================
# --- Feature Separability Analysis Function (Poster - XL Version) ---
# =============================================================================
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
import os

def analyze_feature_separability(features, labels, title_prefix, output_path):
    """
    Performs and plots PCA, LDA (as density curves), and t-SNE.
    Calculates and adds a large d-prime separability score to the LDA plot.
    This version is optimized for extra-large display (e.g., posters).
    """
    print(f"\n--- Analyzing Separability for {title_prefix} ---")
    # Ensure labels are 1D array
    labels = np.array(labels).flatten()

    # Reshape features to 2D (samples, flattened_features)
    if features.ndim > 2:
        features_2d = features.reshape(features.shape[0], -1)
    else:
        features_2d = features

    print(f"Feature shape for analysis: {features_2d.shape}")

    # --- Apply Dimensionality Reduction ---
    pca = PCA(n_components=2)
    features_pca = pca.fit_transform(features_2d)

    lda = LinearDiscriminantAnalysis(n_components=1)
    features_lda = lda.fit_transform(features_2d, labels)

    # Use a smaller subset for t-SNE if the dataset is large to speed it up
    tsne_samples = min(1000, features_2d.shape[0])
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, tsne_samples - 1))

    if features_2d.shape[0] < tsne.perplexity + 1:
        print(f"Warning: Not enough samples ({features_2d.shape[0]}) for t-SNE with perplexity {tsne.perplexity}. Skipping t-SNE.")
        features_tsne = None
        tsne_labels = None
    else:
        features_tsne = tsne.fit_transform(features_2d[:tsne_samples, :])
        tsne_labels = labels[:tsne_samples]

    # --- Plotting ---
    fig, axes = plt.subplots(1, 3, figsize=(36, 12))
    fig.suptitle(f'Feature Separability Analysis: {title_prefix}', fontsize=32, fontweight='bold', y=1.03)

    # Define color, label, and ORDER mappings
    class_label_map = {0: 'Healthy', 1: 'Parkinson'}
    palette = {'Healthy': 'blue', 'Parkinson': 'red'}
    legend_order = ['Healthy', 'Parkinson'] # <<< ADD THIS LINE TO DEFINE ORDER

    # Map numeric labels to string labels for plotting
    mapped_labels = np.array([class_label_map[l] for l in labels])

    # PCA Plot
    sns.scatterplot(x=features_pca[:, 0], y=features_pca[:, 1], hue=mapped_labels,
                    palette=palette, ax=axes[0], s=150, alpha=0.8,
                    hue_order=legend_order) # <<< ADD hue_order HERE
    axes[0].set_title('PCA Projection', fontsize=26)
    axes[0].set_xlabel('Principal Component 1', fontsize=22)
    axes[0].set_ylabel('Principal Component 2', fontsize=22)
    axes[0].tick_params(axis='both', which='major', labelsize=18)
    axes[0].legend(title='Class', fontsize=18, title_fontsize=20)

    # LDA Plot (Density Curves)
    df_lda = pd.DataFrame({'LDA Projection': features_lda.flatten(), 'label': mapped_labels})
    sns.kdeplot(data=df_lda, x='LDA Projection', hue='label', palette=palette,
                ax=axes[1], fill=True, common_norm=False, linewidth=3.5,
                hue_order=legend_order) # <<< ADD hue_order HERE
    axes[1].set_title('LDA Projection (Density)', fontsize=26)
    axes[1].set_xlabel('LDA Projection', fontsize=22)
    axes[1].set_ylabel('Density', fontsize=22)
    axes[1].tick_params(axis='both', which='major', labelsize=18)
    axes[1].legend(title='Class', fontsize=18, title_fontsize=20)


    # Calculate and add d-prime to the LDA plot
    mean_0 = np.mean(features_lda[labels == 0])
    mean_1 = np.mean(features_lda[labels == 1])
    std_0 = np.std(features_lda[labels == 0])
    std_1 = np.std(features_lda[labels == 1])

    if std_0 > 0 and std_1 > 0:
        d_prime = (2 * (mean_1 - mean_0)) / np.sqrt(std_0**2 + std_1**2)
        print(f"Separability Metric (d-prime based on LDA): {d_prime:.4f}")
        axes[1].text(0.05, 0.95, f"d-prime: {d_prime:.4f}",
                     transform=axes[1].transAxes, fontsize=24, fontweight='bold',
                     verticalalignment='top', bbox=dict(boxstyle='round,pad=1.0', fc='yellow', alpha=0.7))
    else:
        print("Could not calculate d-prime (standard deviation is zero for one class).")
        axes[1].text(0.05, 0.95, "d-prime: N/A",
                     transform=axes[1].transAxes, fontsize=24, fontweight='bold',
                     verticalalignment='top', bbox=dict(boxstyle='round,pad=1.0', fc='gray', alpha=0.7))

    # t-SNE Plot
    if features_tsne is not None:
        mapped_tsne_labels = np.array([class_label_map[l] for l in tsne_labels])
        sns.scatterplot(x=features_tsne[:, 0], y=features_tsne[:, 1], hue=mapped_tsne_labels,
                        palette=palette, ax=axes[2], s=150, alpha=0.8,
                        hue_order=legend_order) # <<< ADD hue_order HERE
        axes[2].set_title('t-SNE Projection', fontsize=26)
        axes[2].set_xlabel('t-SNE Dimension 1', fontsize=22)
        axes[2].set_ylabel('t-SNE Dimension 2', fontsize=22)
        axes[2].tick_params(axis='both', which='major', labelsize=18)
        axes[2].legend(title='Class', fontsize=18, title_fontsize=20)
    else:
        axes[2].set_title('t-SNE Projection (Skipped)', fontsize=26)
        axes[2].text(0.5, 0.5, "Not enough samples for t-SNE",
                     transform=axes[2].transAxes, ha='center', va='center', fontsize=18, color='gray')
        axes[2].set_xticks([])
        axes[2].set_yticks([])

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    save_path = os.path.join(output_path, f'{title_prefix.lower().replace(" ", "_")}_separability_analysis_poster_XL.png')
    plt.savefig(save_path, dpi=300)
    plt.show()
    print(f"Extra-large separability plots for poster saved to: {save_path}")

# =============================================================================
# --- Side-by-Side Confusion Matrix and ROC Curve Plotting (Poster Version) ---
# =============================================================================
def plot_cm_and_roc_side_by_side(y_true, y_pred_proba, output_path, threshold=0.5):
    """
    Creates a single, large figure with the enhanced confusion matrix and ROC curve
    plotted side-by-side, suitable for posters and presentations.
    """
    print("\\n--- Generating Poster-Sized Side-by-Side CM and ROC Plot ---")
    y_true = np.array(y_true).flatten()
    y_pred_proba = np.array(y_pred_proba).flatten()
    y_pred_binary = (y_pred_proba > threshold).astype(int)

    # --- Create Larger Figure ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 9))
    fig.suptitle('Model Evaluation: Confusion Matrix & ROC Curve', fontsize=28, fontweight='bold', y=1.02)

    # --- 1. Enhanced Confusion Matrix (Left Subplot) ---
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    labels = np.asarray([
        [f"{tn} ({cm_percent[0,0]:.2%})", f"{fp} ({cm_percent[0,1]:.2%})"],
        [f"{fn} ({cm_percent[1,0]:.2%})", f"{tp} ({cm_percent[1,1]:.2%})"]
    ])
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    sns.heatmap(cm, annot=labels, fmt='', cmap='Blues', cbar=False, ax=ax1,
                xticklabels=['Predicted Healthy', 'Predicted Parkinson'],
                yticklabels=['Actual Healthy', 'Actual Parkinson'],
                annot_kws={"size": 24}) # <<< INCREASED FONT SIZE HERE

    ax1.set_title('Confusion Matrix', fontsize=22)
    ax1.set_xlabel(f'Specificity: {specificity:.4f}', fontsize=18)
    ax1.set_ylabel(f'Sensitivity (Recall): {sensitivity:.4f}', fontsize=18)
    ax1.tick_params(axis='both', which='major', labelsize=16)

    # --- 2. ROC Curve (Right Subplot) ---
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    ax2.plot(fpr, tpr, color='red', lw=3, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    ax2.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    ax2.set_xlim([0.0, 1.0])
    ax2.set_ylim([0.0, 1.0])
    ax2.set_xlabel('False Positive Rate', fontsize=18)
    ax2.set_ylabel('True Positive Rate', fontsize=18)
    ax2.set_title('Receiver Operating Characteristic (ROC) Curve', fontsize=22)
    ax2.legend(loc="lower right", fontsize=16)
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='both', which='major', labelsize=14)

    # --- Save and Show ---
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    save_path = os.path.join(output_path, 'cm_roc_side_by_side_poster.png')
    plt.savefig(save_path, dpi=300)
    plt.show()
    print(f"Poster-sized side-by-side plot saved to: {save_path}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
from sklearn.metrics import confusion_matrix, roc_curve, auc

# =============================================================================
# --- Comprehensive 3-Plot Evaluation (CM, ROC, and Training History) ---
# =============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
from sklearn.metrics import confusion_matrix, roc_curve, auc

# =============================================================================
# --- Comprehensive 3-Plot Evaluation (CM, ROC, and Training History) ---
# =============================================================================
def plot_full_evaluation_square(history, y_true, y_pred_proba, output_path, threshold=0.5):
    """
    Creates a single figure with three plots side-by-side, aiming for a more
    square aspect ratio for each, especially CM and ROC.
    """
    print("\n--- Generating Comprehensive 3-Plot Evaluation Figure (Square Aspect) ---")

    # --- Prepare Data ---
    y_true = np.array(y_true).flatten()
    y_pred_proba = np.array(y_pred_proba).flatten()
    y_pred_binary = (y_pred_proba > threshold).astype(int)
    hist_df = pd.DataFrame(history.history)
    epochs = range(1, len(hist_df) + 1)

    # --- Create Larger 3-Panel Figure with Square Aspect ---
    # We'll use gridspec_kw to ensure the aspect ratio of the first two plots
    # We want each subplot area to be roughly square, so adjust figsize accordingly.
    # For three square plots, a width of ~3*height is a good start.
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(27, 9)) # Adjust figsize, keeping height fixed
    fig.suptitle('Comprehensive Model Evaluation', fontsize=28, fontweight='bold', y=1.02)

    # --- PLOT 1: Enhanced Confusion Matrix (Left) ---
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    labels = np.asarray([
        [f"{tn}\n({cm_percent[0,0]:.2%})", f"{fp}\n({cm_percent[0,1]:.2%})"],
        [f"{fn}\n({cm_percent[1,0]:.2%})", f"{tp}\n({cm_percent[1,1]:.2%})"]
    ])
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    sns.heatmap(cm, annot=labels, fmt='', cmap='Blues', cbar=False, ax=ax1,
                xticklabels=['Predicted Healthy', 'Predicted Parkinson'],
                yticklabels=['Actual Healthy', 'Actual Parkinson'],
                annot_kws={"size": 24})
    ax1.set_title('Confusion Matrix', fontsize=22)
    ax1.set_xlabel(f'Specificity: {specificity:.4f}', fontsize=18)
    ax1.set_ylabel(f'Sensitivity (Recall): {sensitivity:.4f}', fontsize=18)
    ax1.tick_params(axis='both', which='major', labelsize=16)
    ax1.set_aspect('equal', adjustable='box') # Force square aspect

    # --- PLOT 2: ROC Curve (Middle) ---
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    ax2.plot(fpr, tpr, color='red', lw=3, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    ax2.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    ax2.set_xlim([0.0, 1.0])
    ax2.set_ylim([0.0, 1.0])
    ax2.set_xlabel('False Positive Rate', fontsize=18)
    ax2.set_ylabel('True Positive Rate', fontsize=18)
    ax2.set_title('Receiver Operating Characteristic', fontsize=22)
    ax2.legend(loc="lower right", fontsize=16)
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='both', which='major', labelsize=14)
    ax2.set_aspect('equal', adjustable='box') # Force square aspect

    # --- PLOT 3: Training History (Right) ---
    # For the history plot, we maintain its current aspect as forcing 'equal'
    # would distort the time-series nature of loss and AUC.
    # Instead, we ensure its overall subplot area is square, and its internal
    # content will fill that area appropriately.
    ax3.set_title('Training History', fontsize=22)
    ax3.set_xlabel('Epoch', fontsize=18)

    # Plot Loss on the primary y-axis
    ax3_loss = ax3
    color = 'tab:red'
    ax3_loss.set_ylabel('Loss', color=color, fontsize=18)
    p1 = ax3_loss.plot(epochs, hist_df['loss'], color=color, linestyle='-', label='Training Loss')
    p2 = ax3_loss.plot(epochs, hist_df['val_loss'], color=color, linestyle='--', label='Validation Loss')
    ax3_loss.tick_params(axis='y', labelcolor=color, labelsize=14)

    # Plot AUC on the secondary y-axis
    ax3_auc = ax3.twinx()
    color = 'tab:blue'
    ax3_auc.set_ylabel('AUC', color=color, fontsize=18)
    p3 = ax3_auc.plot(epochs, hist_df['auc'], color=color, linestyle='-', label='Training AUC')
    p4 = ax3_auc.plot(epochs, hist_df['val_auc'], color=color, linestyle='--', label='Validation AUC')
    ax3_auc.tick_params(axis='y', labelcolor=color, labelsize=14)

    # Unified legend for the third plot
    lines = p1 + p2 + p3 + p4
    labels = [l.get_label() for l in lines]
    ax3.legend(lines, labels, loc='center right', fontsize=14)
    ax3.grid(True, alpha=0.3)
    # ax3.set_aspect('equal', adjustable='box') # Only if you truly want to distort it. Not recommended here.

    # --- Save and Show ---
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    save_path = os.path.join(output_path, 'full_evaluation_report_square.png')
    plt.savefig(save_path, dpi=300)
    plt.show()
    print(f"Comprehensive evaluation plot (square aspect) saved to: {save_path}")

### Model Explainability: Interpreting Decisions

To move beyond "black box" predictions, this module implements techniques to visualize *what* the model is looking at. It specifically helps diagnose issues like "tunnel vision" (where a model might over-focus on a recording artifact rather than the voice itself).

#### 1. Feature Map Layout (`generate_feature_map_info`)
Since the input to the model is a concatenated matrix of different features (e.g., Mel Spectrogram rows 0-29, MFCC rows 30-59), we need a "map" to interpret the results.
* **Introspection:** It loads the original `.npz` feature file and inspects the shapes of the arrays inside.
* **Layout Generation:** It dynamically creates a color-coded mask that maps specific rows of the input matrix back to their feature names (e.g., "Rows 0-30 are Mel Spectrogram").
* **Visualization:** This allows us to overlay a legend on our heatmaps, clearly showing which biological feature corresponds to a high-attention region.

#### 2. Grad-CAM Analysis (`run_gradcam_analysis`)
This function performs **Gradient-weighted Class Activation Mapping** to visualize the spatial attention of the CNN branch.

* **Backpropagation Logic:**
    1.  It selects a random subset of **True Positive (Parkinson's)** and **True Negative (Healthy)** samples.
    2.  For each sample, it computes the gradient of the predicted class score with respect to the feature maps of the *last convolutional layer* (`conv2b`).
    3.  These gradients act as weights: high gradients mean that specific region was crucial for the decision.
    4.  The weighted feature maps are summed to create a coarse "heatmap."

* **Upscaling:** The coarse heatmap (from the deep, pooled layer) is resized back to the original input dimensions (`194 x 60`) so it can be overlaid on the input features.

* **Visualization (The 3-Panel Plot):**
    * **Panel 1 (PD Attention):** Average heatmap for Parkinson's patients. Hot spots (red) show where the model looks to find disease indicators.
    * **Panel 2 (Healthy Attention):** Average heatmap for healthy controls.
    * **Panel 3 (Feature Map):** The color-coded legend generated earlier, transposed to match the orientation of the heatmaps (Time on Y-axis, Features on X-axis).

**Why this matters:** If the "PD Attention" map shows a distinct vertical line at the very start of the audio, it proves the model is suffering from the "tunnel vision" artifact described in the thesis.

In [ ]:
# =============================================================================
# --- Model Explainability (SHAP & Grad-CAM) ---
# =============================================================================

def generate_feature_map_info(npz_file_path):
    """
    Dynamically generates feature layout information (color mask, names, colors)
    by inspecting the contents of an .npz file.
    """
    if not os.path.exists(npz_file_path):
        print(f"Error: File not found at '{npz_file_path}'")
        return None

    feature_layout = {}
    print(f"Inspecting file for feature map info: {npz_file_path}")

    with np.load(npz_file_path) as data:
        ignore_keys = {'labels', 'sex', 'age', 'X', 'y'}
        feature_keys = [key for key in data.files if key not in ignore_keys]

        if not feature_keys:
            print("Error: No feature arrays found in the .npz file.")
            return None

        for key in sorted(feature_keys):
            arr_shape = data[key].shape

            if len(arr_shape) == 1:
                num_rows = 1
            elif len(arr_shape) >= 2:
                num_rows = arr_shape[1]
            else:
                continue

            feature_layout[key] = num_rows

    colors = plt.get_cmap('Paired', len(feature_layout))
    feature_names = list(feature_layout.keys())
    total_rows = sum(feature_layout.values())
    time_steps_for_visual = total_rows

    color_mask = np.zeros((total_rows, time_steps_for_visual), dtype=int)

    current_row = 0
    for i, (name, num_rows) in enumerate(feature_layout.items()):
        color_mask[current_row : current_row + num_rows, :] = i
        current_row += num_rows

    legend_patches = [mpatches.Patch(color=colors(i), label=f"{name} ({feature_layout[name]} rows)")
                      for i, name in enumerate(feature_names)]

    return {
        'color_mask': color_mask,
        'feature_layout': feature_layout,
        'feature_names': feature_names,
        'colors': colors,
        'legend_patches': legend_patches,
        'total_rows': total_rows
    }

def run_gradcam_analysis(model, X_test, y_test, output_path, FEATURES_FILE_PATH, num_samples = 50):
    """
    Run Grad-CAM analysis and plots the average heatmaps for PD and HC alongside the
    feature map. The plots are transposed to match SHAP's orientation.
    """
    print("\n--- Running Grad-CAM Analysis (3-Plot Version) ---")
    os.makedirs(output_path, exist_ok=True)

    parkinson_detector = None
    for layer in model.layers:
        if 'ParkinsonDetectorModel' in str(type(layer)):
            parkinson_detector = layer
            break

    if parkinson_detector is None:
        print("ParkinsonDetectorModel not found in the model layers.")
        return

    # --- (This entire section for calculating heatmaps is unchanged) ---
    def get_conv_and_output(inputs):
        x = parkinson_detector.reshape_in(inputs)
        x = parkinson_detector.conv1a(x)
        x = parkinson_detector.conv1b(x)
        x = parkinson_detector.pool1(x)
        x = parkinson_detector.drop1(x, training=False)
        x = parkinson_detector.conv2a(x)
        conv_output = parkinson_detector.conv2b(x)
        x = parkinson_detector.pool2(conv_output)
        x = parkinson_detector.drop2(x, training=False)
        cnn_flat = parkinson_detector.flatten_cnn(x)
        shape = tf.shape(x)
        sequence = tf.reshape(x, [-1, shape[1] * shape[2], shape[3]])
        att_out = parkinson_detector.attention(query=sequence, key=sequence, value=sequence)
        att_flat = parkinson_detector.flatten_att(att_out)
        lstm_seq = parkinson_detector.lstm1(sequence)
        lstm_out = parkinson_detector.lstm2(lstm_seq)
        lstm_out = parkinson_detector.drop_lstm(lstm_out, training=False)
        concatenated = parkinson_detector.concat([cnn_flat, att_flat, lstm_out])
        bottleneck = parkinson_detector.dense_bottleneck(concatenated)
        final_output = parkinson_detector.dense_output(bottleneck)
        return conv_output, final_output

    parkinson_indices = np.where(y_test == 1)[0]
    healthy_indices = np.where(y_test == 0)[0]
    selected_pd_indices = list(np.random.choice(parkinson_indices, min(num_samples, len(parkinson_indices)), replace=False))
    selected_hc_indices = list(np.random.choice(healthy_indices, min(num_samples, len(healthy_indices)), replace=False))
    print(f"Selected {len(selected_pd_indices)} PD and {len(selected_hc_indices)} HC samples for Grad-CAM.")

    tp_heatmaps, tn_heatmaps = [], []
    for i in tqdm(selected_pd_indices, desc="Grad-CAM PD Progress"):
        img = X_test[i:i+1]
        with tf.GradientTape() as tape:
            img_tensor = tf.cast(img, tf.float32)
            tape.watch(img_tensor)
            conv_outputs, preds = get_conv_and_output(img_tensor)
            loss = preds[:, 0]
        grads = tape.gradient(loss, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        conv_outputs_np = conv_outputs[0].numpy()
        pooled_grads_np = pooled_grads.numpy()
        heatmap = np.sum(pooled_grads_np * conv_outputs_np, axis=-1)
        heatmap = np.maximum(heatmap, 0)
        heatmap /= (heatmap.max() + 1e-10)
        tp_heatmaps.append(heatmap)

    for i in tqdm(selected_hc_indices, desc="Grad-CAM HC Progress"):
        img = X_test[i:i+1]
        with tf.GradientTape() as tape:
            img_tensor = tf.cast(img, tf.float32)
            tape.watch(img_tensor)
            conv_outputs, preds = get_conv_and_output(img_tensor)
            loss = preds[:, 0]
        grads = tape.gradient(loss, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        conv_outputs_np = conv_outputs[0].numpy()
        pooled_grads_np = pooled_grads.numpy()
        heatmap = np.sum(pooled_grads_np * conv_outputs_np, axis=-1)
        heatmap = np.maximum(heatmap, 0)
        heatmap /= (heatmap.max() + 1e-10)
        tn_heatmaps.append(heatmap)

    avg_pd_heatmap = np.mean(tp_heatmaps, axis=0) if tp_heatmaps else np.zeros((X_test.shape[1], X_test.shape[2]))
    avg_hc_heatmap = np.mean(tn_heatmaps, axis=0) if tn_heatmaps else np.zeros((X_test.shape[1], X_test.shape[2]))
    original_feature_rows, original_time_steps = X_test.shape[1], X_test.shape[2]
    upscaled_avg_pd_heatmap = resize(avg_pd_heatmap, (original_feature_rows, original_time_steps), order=3, mode='reflect', anti_aliasing=True)
    upscaled_avg_hc_heatmap = resize(avg_hc_heatmap, (original_feature_rows, original_time_steps), order=3, mode='reflect', anti_aliasing=True)
    # --- (End of unchanged section) ---

    # --- Plotting: 3 plots in a single row (WITH SWAPPED AXES) ---
    fig, axes = plt.subplots(1, 3, figsize=(18, 7)) # <<< MODIFIED: Changed to 3 subplots
    fig.suptitle("Grad-CAM: Model Attention and Feature Layout", fontsize=16, fontweight='bold')

    # --- 1. Parkinson's - Upscaled Heatmap (Transposed) ---
    ax_pd_heatmap = axes[0]
    im_pd_hm = ax_pd_heatmap.imshow(upscaled_avg_pd_heatmap.T, cmap='jet', aspect='auto', origin='upper')
    ax_pd_heatmap.set_title(f'Avg. PD Attention\n({len(selected_pd_indices)} samples)')
    ax_pd_heatmap.set_ylabel("Time Steps") # <<< MODIFIED
    ax_pd_heatmap.set_xlabel("Feature Rows") # <<< MODIFIED
    divider_pd = make_axes_locatable(ax_pd_heatmap)
    cax_pd = divider_pd.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(im_pd_hm, cax=cax_pd)

    # --- 2. Healthy - Upscaled Heatmap (Transposed) ---
    ax_hc_heatmap = axes[1]
    im_hc_hm = ax_hc_heatmap.imshow(upscaled_avg_hc_heatmap.T, cmap='jet', aspect='auto', origin='upper')
    ax_hc_heatmap.set_title(f'Avg. Healthy Attention\n({len(selected_hc_indices)} samples)')
    ax_hc_heatmap.set_xlabel("Feature Rows") # <<< MODIFIED
    ax_hc_heatmap.set_yticklabels([]) # Hide y-axis labels to avoid repetition
    divider_hc = make_axes_locatable(ax_hc_heatmap)
    cax_hc = divider_hc.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(im_hc_hm, cax=cax_hc)

    # --- 3. Feature Map (Transposed) ---
    ax_feature_map = axes[2]
    feature_map_info = generate_feature_map_info(FEATURES_FILE_PATH)
    if feature_map_info:
        legend_colors = [patch.get_facecolor() for patch in feature_map_info['legend_patches']]
        cmap = ListedColormap(legend_colors)
        transposed_data = feature_map_info['color_mask'].T
        im_feat = ax_feature_map.imshow(transposed_data, cmap=cmap, aspect='auto', interpolation='nearest', origin='upper')
        ax_feature_map.set_title("Feature Layout")
        ax_feature_map.set_xlabel("Features")
        ax_feature_map.set_yticklabels([]) # Hide y-axis labels
        fig.legend(handles=feature_map_info['legend_patches'], loc='center left', bbox_to_anchor=(0.93, 0.5), borderaxespad=0.)
    else:
        ax_feature_map.set_visible(False)

    # --- Final Save ---
    plt.tight_layout(rect=[0, 0, 0.93, 0.93])
    save_path = os.path.join(output_path, "gradcam_attention_comparison.png") # <<< MODIFIED filename
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved 3-plot Grad-CAM comparison to {save_path}")


### SHAP (SHapley Additive exPlanations) Analysis Pipeline

This function (`run_full_shap_analysis`) is the core of the explainability module. It leverages SHAP to explain *why* the model makes specific predictions by assigning an importance value to every input feature (time-frequency bin).

#### 1. Configuration & Feature Mapping
* **Feature Layout:** Before running SHAP, the function calls `generate_feature_map_info` (assumed to be defined elsewhere) to understand the structure of the input data (e.g., which rows correspond to Mel Spectrograms vs. MFCCs). This allows for human-readable labels later.
* **Helper Function `feature_name_for_row`:** Maps a numerical row index (e.g., row 15) to its feature name (e.g., "Mel Spectrogram").

#### 2. Balanced Sample Selection
To ensure the explanation is not biased toward the majority class:
* The function identifies indices for **Healthy Controls (HC)** and **Parkinson's (PD)** samples in the test set.
* It randomly selects a balanced number of samples (default `samples_per_class=50`) from each group.
* These samples are concatenated and shuffled to form the `test_samples` batch for explanation.

#### 3. SHAP Value Calculation
* **Explainer:** Uses `shap.GradientExplainer`, which is efficient for deep learning models. It requires a background dataset (`X_train[:50]`) to serve as a reference point for "zero" feature contribution.
* **Computation:** Iterates through the `test_samples` batch, computing SHAP values for each sample. The result is a 3D array: `(n_samples, n_features, n_time_steps)`.

#### 4. Visualization 1: Global Feature Importance (Bar Chart)
* **Ranking:** The function calculates the mean absolute SHAP value for every feature across all samples. This identifies the most influential features globally.
* **Plotting:** A bar chart displays the top 20 most important features. Labels are constructed dynamically (e.g., "Mel Spectrogram T10") to pinpoint exactly *what* and *when* the model is looking at.

#### 5. Visualization 2: Local & Comparative Heatmaps
The function generates detailed heatmaps to compare how the model views Healthy vs. Parkinson's samples.
* **Average HC/PD Maps:** Computes the average SHAP values for all Healthy samples and all Parkinson's samples separately.
* **Difference Map (PD - HC):** This is the most critical visualization. It subtracts the Healthy map from the Parkinson's map.
    * **Red Regions:** Features that push the prediction *towards* Parkinson's.
    * **Blue Regions:** Features that push the prediction *towards* Healthy.
    * **Insight:** This map often reveals the specific biomarkers (e.g., tremors at specific frequencies) the model has learned.
* **Transposed Display:** The plotting helper `plot_aligned_heatmap` swaps axes so that **Time** is on the Y-axis and **Features** are on the X-axis, matching standard spectrogram layouts.

#### 6. Combined Output
Finally, a comprehensive figure is assembled:
* **Left:** Global Feature Importance Bar Chart.
* **Right:** A 2x2 grid containing the Average HC heatmap, Average PD heatmap, Difference heatmap, and a reference Feature Map legend.
* **Statistical Significance (Optional):** If enough samples are available, a t-test is performed to generate a p-value map, highlighting which feature differences are statistically significant.

In [ ]:
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
from tqdm import tqdm
from scipy.stats import ttest_ind
import shap
from tensorflow.keras.layers import Lambda, Reshape

def feature_name_for_row(row_idx, feature_layout):
    """
    feature_layout: ordered dict-like mapping feature_name -> num_rows
    returns the feature_name that contains the given row_idx (0-based).
    """
    cum = 0
    for name, nrows in feature_layout.items():
        if row_idx < cum + nrows:
            return name
        cum += nrows
    return "Unknown"

def run_full_shap_analysis(model, X_train, X_test, y_test, output_path, features_npz_path, samples_per_class = 50, top_n=20):
    """
    Run SHAP analysis with balanced sample selection.
    Produces:
      - combined figure: left = global top-N bar (with feature-category labels),
                         right = 2x2 grid with average HC, average PD, diff (PD-HC) and feature map.
      - also saves per-figure PNGs for backward compatibility.
    """
    print("\n--- Running Full SHAP Analysis (updated plotting) ---")
    os.makedirs(output_path, exist_ok=True)

    feature_map_info = generate_feature_map_info(features_npz_path)
    if feature_map_info is None:
        return

    legend_patches = feature_map_info['legend_patches']
    total_rows = feature_map_info['total_rows']

    # Balanced sample selection
    healthy_indices = np.where(y_test == 0)[0]
    parkinson_indices = np.where(y_test == 1)[0]
    num_healthy_to_select = min(samples_per_class, len(healthy_indices))
    num_parkinson_to_select = min(samples_per_class, len(parkinson_indices))
    print(f"Attempting to select {num_healthy_to_select} healthy and {num_parkinson_to_select} Parkinson's samples.")
    selected_healthy_indices = np.random.choice(healthy_indices, num_healthy_to_select, replace=False) if num_healthy_to_select>0 else np.array([])
    selected_parkinson_indices = np.random.choice(parkinson_indices, num_parkinson_to_select, replace=False) if num_parkinson_to_select>0 else np.array([])
    final_indices = np.concatenate([selected_healthy_indices, selected_parkinson_indices]).astype(int)
    np.random.shuffle(final_indices)
    test_samples = X_test[final_indices]
    y_true_samples = y_test[final_indices]

    print(f"Calculating SHAP values for {len(test_samples)} balanced samples...")
    explainer = shap.GradientExplainer(model, X_train[:50].astype(np.float32))
    shap_values_list = []
    for sample in tqdm(test_samples, desc="SHAP Progress"):
        sample_batch = np.expand_dims(sample, axis=0).astype(np.float32)
        sv = explainer.shap_values(sample_batch)
        if isinstance(sv, list): sv = sv[0]
        shap_values_list.append(sv)
    shap_values = np.vstack(shap_values_list)  # shape: (n_samples, rows, time)
    print(f"\nSHAP values shape: {shap_values.shape}, Test samples shape: {test_samples.shape}")
    if shap_values.shape[1] != total_rows:
        print(f"⚠️ Warning: SHAP values feature rows ({shap_values.shape[1]}) do not match total_rows from NPZ ({total_rows}).")
    actual_data_time_steps = shap_values.shape[2]

    # --- Global Top-N bar with feature-category labels ---
    flat_shap = shap_values.reshape(shap_values.shape[0], -1)
    mean_abs = np.mean(np.abs(flat_shap), axis=0)
    top_idx = np.argsort(mean_abs)[::-1][:top_n]
    coords = [np.unravel_index(i, (shap_values.shape[1], shap_values.shape[2])) for i in top_idx]  # (row, time)

    # Build labels using feature_map_info
    feature_layout = feature_map_info['feature_layout']
    # ensure ordered iteration like in generate_feature_map_info
    if isinstance(feature_layout, dict):
        # ok: keep insertion order
        pass

    labels = []
    for row_idx, t_idx in coords:
        fname = feature_name_for_row(row_idx, feature_layout)
        # use 1-based time index for readability
        labels.append(f"{fname} T{t_idx+1}")

    # save standalone global bar
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(top_idx)), mean_abs[top_idx])
    plt.xticks(range(len(top_idx)), labels, rotation=45, ha="right")
    plt.title(f"Top-{top_n} Global SHAP Features (by mean absolute SHAP)")
    plt.xlabel("Feature Category × Time")
    plt.ylabel("Mean |SHAP value|")
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, "shap_global_bar.png"), dpi=300, bbox_inches="tight")
    plt.close()
    print("-> Saved 'shap_global_bar.png'")

    # --- helper plotting: aligned heatmap (but with axes swapped/transposed so x/y are reversed consistently) ---
    def plot_aligned_heatmap(heatmap_data, title, filename_suffix, cmap, label, vmin=None, vmax=None, ax=None):
        """
        Plots the SHAP heatmap and Feature Map side by side, BUT displays them with swapped axes
        (so time runs on y-axis and features on x-axis in the displayed figure) — consistent with request to invert axes.
        If ax provided, use it for the heatmap; otherwise create a new fig/ax pair and save to disk.
        """
        # transpose to swap axes for display (rows,time) -> (time,rows)
        disp_data = heatmap_data.T  # now shape = (time, rows) -> will show time on vertical, features on horizontal

        # If ax not given, create fig with both maps; otherwise only plot into provided ax (featuremap handled separately)
        created_fig = False
        if ax is None:
            fig, axes = plt.subplots(1, 2, figsize=(14, 7))
            ax_shap, ax_feature_map_sub = axes[0], axes[1]
            created_fig = True
        else:
            ax_shap = ax
            # user must plot feature map separately in combined figure

        img = ax_shap.imshow(disp_data, cmap=cmap, aspect='auto', interpolation='nearest', vmin=vmin, vmax=vmax)
        ax_shap.set_title(title, fontsize=10)
        # since we transposed, x-axis is "Features (rows)", y-axis is "Time steps"
        ax_shap.set_xlabel(f"Feature Rows ({total_rows})", fontsize=9)
        ax_shap.set_ylabel(f"Time Steps (Actual: {actual_data_time_steps})", fontsize=9)
        # invert axes visually if you still want reversed direction (optional). Commented out but left for clarity:
        # ax_shap.invert_xaxis(); ax_shap.invert_yaxis()

        if created_fig:
            divider = make_axes_locatable(ax_shap)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            fig.colorbar(img, cax=cax, label=label)

            # --- Feature Map on right (aligned, transposed to match display orientation) ---
            aligned_color_mask = np.zeros((total_rows, actual_data_time_steps), dtype=int)
            current_row = 0
            for i, (name, num_rows) in enumerate(feature_map_info['feature_layout'].items()):
                aligned_color_mask[current_row : current_row + num_rows, :] = i
                current_row += num_rows
            # transpose to show time x features consistent with heatmap display
            transposed_mask = aligned_color_mask.T
            legend_colors = [patch.get_facecolor() for patch in feature_map_info['legend_patches']]
            cmap_feat = ListedColormap(legend_colors)
            ax_feature_map_sub.imshow(transposed_mask, cmap=cmap_feat, aspect='auto', interpolation='nearest')
            ax_feature_map_sub.set_title("Feature Map (transposed)", fontsize=10)
            ax_feature_map_sub.set_xlabel("Feature Index")
            ax_feature_map_sub.set_ylabel("Time Steps")
            ax_feature_map_sub.tick_params(axis='x', labelbottom=False)
            fig.legend(handles=feature_map_info['legend_patches'], loc='center left', bbox_to_anchor=(0.93, 0.5), fontsize=8)
            fig.suptitle(f"SHAP Analysis: {title}", fontsize=14, fontweight='bold')
            plt.tight_layout(rect=[0, 0, 0.93, 0.95])
            plt.savefig(os.path.join(output_path, f"shap_aligned_{filename_suffix}.png"), dpi=300)
            plt.close(fig)
            print(f"-> Saved 'shap_aligned_{filename_suffix}.png'")
        else:
            return img

    # compute masks and means
    hc_mask, pd_mask = (y_true_samples == 0), (y_true_samples == 1)
    hc_mean = None; pd_mean = None
    if np.any(hc_mask):
        hc_mean = shap_values[hc_mask].mean(axis=0).squeeze()
    if np.any(pd_mask):
        pd_mean = shap_values[pd_mask].mean(axis=0).squeeze()

    # --- Combined figure: left = global bar, right = 2x2 with HC, PD, Diff, FeatureMap ---
    import matplotlib.gridspec as gridspec
    fig = plt.figure(constrained_layout=False, figsize=(18, 10))
    gs = gridspec.GridSpec(2, 3, figure=fig, width_ratios=[1.0, 0.6, 0.6], height_ratios=[1,1], wspace=0.3, hspace=0.25)

    # Left: vertical global bar (occupies both rows)
    ax_bar = fig.add_subplot(gs[:, 0])
    ax_bar.bar(range(len(top_idx)), mean_abs[top_idx])
    ax_bar.set_xticks(range(len(top_idx)))
    ax_bar.set_xticklabels(labels, rotation=45, ha='right')
    ax_bar.set_title(f"Top-{top_n} Global SHAP Features")
    ax_bar.set_ylabel("Mean |SHAP value|")

    # Right-top: Average Healthy
    ax_hc = fig.add_subplot(gs[0, 1])
    if hc_mean is not None:
        # pass ax to plot_aligned_heatmap to only draw the heatmap there (featuremap will be separate)
        img_hc = plot_aligned_heatmap(hc_mean, "Average SHAP - Healthy", "tmp_hc", "bwr", "Mean SHAP", ax=ax_hc)
        # add colorbar for this axis
        divider = make_axes_locatable(ax_hc)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(img_hc, cax=cax)
    else:
        ax_hc.text(0.5, 0.5, 'No HC samples', ha='center', va='center', transform=ax_hc.transAxes)

    # Right-middle: Average Parkinson
    ax_pd = fig.add_subplot(gs[0, 2])
    if pd_mean is not None:
        img_pd = plot_aligned_heatmap(pd_mean, "Average SHAP - Parkinson", "tmp_pd", "bwr", "Mean SHAP", ax=ax_pd)
        divider = make_axes_locatable(ax_pd)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(img_pd, cax=cax)
    else:
        ax_pd.text(0.5, 0.5, 'No PD samples', ha='center', va='center', transform=ax_pd.transAxes)

    # Right-bottom-left: Difference map (PD - HC)
    ax_diff = fig.add_subplot(gs[1, 1])
    if (hc_mean is not None) and (pd_mean is not None):
        diff_map = pd_mean - hc_mean
        max_abs_diff = np.max(np.abs(diff_map))
        img_diff = plot_aligned_heatmap(diff_map, "SHAP Diff (PD - HC)", "tmp_diff", "seismic", "Δ SHAP", vmin=-max_abs_diff, vmax=max_abs_diff, ax=ax_diff)
        divider = make_axes_locatable(ax_diff)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(img_diff, cax=cax)
    else:
        ax_diff.text(0.5, 0.5, 'Insufficient data for diff', ha='center', va='center', transform=ax_diff.transAxes)

    # Right-bottom-right: Feature map (transposed to match heatmaps)
    ax_feat = fig.add_subplot(gs[1, 2])
    aligned_color_mask = np.zeros((total_rows, actual_data_time_steps), dtype=int)
    current_row = 0
    for i, (name, num_rows) in enumerate(feature_map_info['feature_layout'].items()):
        aligned_color_mask[current_row : current_row + num_rows, :] = i
        current_row += num_rows
    transposed_mask = aligned_color_mask.T
    legend_colors = [patch.get_facecolor() for patch in feature_map_info['legend_patches']]
    cmap_feat = ListedColormap(legend_colors)
    im_feat = ax_feat.imshow(transposed_mask, cmap=cmap_feat, aspect='auto', interpolation='nearest')
    ax_feat.set_title("Feature Map (transposed)", fontsize=10)
    ax_feat.set_xlabel("Feature Index")
    ax_feat.set_ylabel("Time Steps")
    # Legend on right of figure
    fig.legend(handles=feature_map_info['legend_patches'], loc='center left', bbox_to_anchor=(0.93, 0.5), fontsize=8)

    # save combined
    plt.suptitle("SHAP Global + Local (with Feature Map)", fontsize=16, fontweight='bold')
    plt.savefig(os.path.join(output_path, "shap_combined_global_local.png"), dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("-> Saved 'shap_combined_global_local.png'")

    # optional: generate significance map if enough samples
    if np.any(hc_mask) and np.any(pd_mask) and np.sum(hc_mask) > 1 and np.sum(pd_mask) > 1:
        hc_vals = shap_values[hc_mask].reshape(np.sum(hc_mask), -1)
        pd_vals = shap_values[pd_mask].reshape(np.sum(pd_mask), -1)
        t_stat, p_vals = ttest_ind(pd_vals, hc_vals, axis=0, equal_var=False)
        p_map = p_vals.reshape(shap_values.shape[1], shap_values.shape[2])
        plot_aligned_heatmap(p_map, "Statistical Significance Map (PD vs HC)", "significance", "viridis_r", "p-value (t-test)", vmin=0, vmax=0.05)
    else:
        print("Skipping significance map: Not enough samples for t-test.")

    print("\n--- SHAP Analysis Complete ---")


### Main Execution Pipeline

This section orchestrates the end-to-end workflow, connecting data loading, model training, evaluation, and post-hoc analysis.

#### 1. Data Preparation
* **Loading:** Reads the pre-processed feature matrix (`.npz`).
* **Splitting:** Performs a **stratified train-test split (80/20)**. Stratification is crucial here to ensure that the ratio of Healthy to Parkinson's patients remains consistent in both sets, preventing bias.

#### 2. Model Training
* **Initialization:** Builds the hybrid `ParkinsonDetectorModel` based on the input shape.
* **Compilation:** Configures the **Adam** optimizer (`lr=0.001`) and **Binary Cross-Entropy** loss. Tracks Accuracy and AUC.
* **Fitting:** Trains for **30 epochs** with a batch size of 32. Uses the `checkpoint_cb` to save only the best model weights based on validation performance.

#### 3. Feature Space Analysis
To validate the model's learning capability, the script performs a "Separability Analysis" at two stages:
1.  **Input Features:** Analyzes the raw data before processing. Usually shows high overlap between classes.
2.  **Dense Layer Inputs:** Extracts the learned representations just before the final classification layer. This typically demonstrates how the model has clustered the classes, aiming for a high **d-prime** separation score.

#### 4. Evaluation & Visualization
* **Metrics:** Calculates predictions and saves raw probabilities to `evaluation.csv`.
* **Plotting:** Generates a comprehensive square-aspect figure containing the Confusion Matrix, ROC Curve, and Training History (Loss/AUC curves) for a complete performance snapshot.

#### 5. Explainability (XAI)
Finally, the script loads the best saved model state to run computationally expensive interpretability checks:
* **SHAP:** quantifies feature importance.
* **Grad-CAM:** visualizes spatial attention maps.

In [ ]:
# =============================================================================
# --- Main Execution ---
# =============================================================================
if __name__ == '__main__':

    X, y= load_data(FEATURES_FILE_PATH)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print(f"\nData split into training ({len(y_train)}) and testing ({len(y_test)}) sets.")

    model = build_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    model.summary()
    optimizer = Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

    print("\n--- Starting model training ---")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[checkpoint_cb],
        verbose=1
    )
    print("--- Model training finished ---")

    pd.DataFrame(history.history).to_csv(HISTORY_SAVE_PATH, index_label='epoch')
    print(f"\nTraining history saved to '{HISTORY_SAVE_PATH}'")
    available_samples = len(X_test)
    requested_samples = min(200, available_samples)  # Use the smaller of the two
    extract_dense_layer_input_features(model, X_test, y_test, MODEL_PATH, requested_samples)


    print("\n--- Start evaluating modrl ---")
    y_pred_probabilities = model.predict(X_test)
    save_metrics_to_csv(y_test, y_pred_probabilities, EVALUATION_FILE_PATH)
    # --- 2. Analyze Input Feature Separability ---
    # This uses the original X_test data before it's fed to the model
    analyze_feature_separability(
        features=X_test,
        labels=y_test,
        title_prefix="Input Features",
        output_path=MODEL_PATH
    )

    # --- 3. Analyze Dense Layer Input Feature Separability ---
    # First, load the features that you extracted earlier
    dense_features_path = os.path.join(MODEL_PATH, "dense_layer_input_features.npz")
    if os.path.exists(dense_features_path):
        with np.load(dense_features_path) as data:
            dense_features = data['concatenated_features']
            dense_labels = data['labels']

        # Now, run the analysis on these extracted features
        analyze_feature_separability(
            features=dense_features,
            labels=dense_labels,
            title_prefix="Dense Layer Input Features",
            output_path=MODEL_PATH
        )
    else:
        print(f"\\n⚠️  Warning: Dense layer features not found at {dense_features_path}. Skipping analysis.")

    print("\\n" + "="*80)
    print("✅ ENHANCED ANALYSIS COMPLETE")
    print("="*80)
        # *** CALL THE NEW ALL-IN-ONE FUNCTION HERE ***
    plot_full_evaluation_square(history, y_test, y_pred_probabilities, MODEL_PATH)

    if os.path.exists(BEST_MODEL_PATH):
        print("\n--- Loading best saved model for explainability analysis ---")
        best_model = load_model(BEST_MODEL_PATH, custom_objects={'ParkinsonDetectorModel': ParkinsonDetectorModel})
        run_full_shap_analysis(model, X_train, X_test, y_test, SHAP_OUTPUT_PATH, FEATURES_FILE_PATH, 50, 20)

    run_gradcam_analysis(best_model, X_test, y_test, GRADCAM_OUTPUT_PATH, FEATURES_FILE_PATH, 50)

### End-to-End Evaluation on Raw Audio

This script provides the final "sanity check" for the trained model. Unlike the previous scripts that operate on pre-processed feature matrices (`.npz` files), this script simulates a real-world deployment scenario. It loads the saved model and runs it directly against raw `.wav` audio files stored in folders, performing all necessary pre-processing on the fly.

#### 1. Configuration & Constants
It re-defines the critical Digital Signal Processing (DSP) parameters used during training (`SAMPLE_RATE`, `DURATION_S`, `N_MELS`, etc.). It is crucial that these match the training phase exactly; otherwise, the input shape to the model will be incorrect.

#### 2. Model Loading
It loads the best-performing model (saved as `best_model.keras`) from the results directory. This confirms that the saved model file is valid and can be deserialized correctly.

#### 3. Real-Time Processing Function (`process_and_predict`)
This is the core inference engine. For a single raw audio file:
1.  **Load:** Reads the audio using `librosa`.
2.  **Fix Length:** Pads or trims the audio to exactly 3 seconds.
3.  **Feature Extraction:** Computes the Mel Spectrogram and MFCCs.
4.  **Formatting:** Stacks the features vertically and adds a batch dimension (shape becomes `(1, 194, 60)`).
5.  **Prediction:** Passes this tensor to the model to get a probability score (0.0 to 1.0).

#### 4. Batch Processing (`test_files_in_folder`)
Iterates through all `.wav` files in a specific directory (e.g., `healthy_control`), runs the inference function on each, and collects the true labels and predicted probabilities.

#### 5. Evaluation & Visualization
Once predictions are gathered for both healthy and patient folders:
* **Metrics:** Calculates Accuracy, Precision, Recall, and F1-score and saves them to `evaluation_on_raw_audio.csv`.
* **Poster-Quality Plot:** Uses `plot_enhanced_confusion_matrix` to generate a high-resolution, aesthetically polished confusion matrix. This plot includes percentages and key metrics (Sensitivity/Specificity) directly on the axes, making it ideal for inclusion in research posters or papers.

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras

# --- 1. Audio and Feature Extraction Parameters ---
SAMPLE_RATE = 16000
DURATION_S = 3
AUDIO_SAMPLES = SAMPLE_RATE * DURATION_S
N_MELS = 30
N_MFCC = 30
N_FFT = 2048
HOP_LENGTH = 512
EXPECTED_FRAMES = int(np.ceil(AUDIO_SAMPLES / HOP_LENGTH))

# --- 2. Paths to Model and Data ---
MODEL_PATH = os.path.join(os.getcwd(), dataset, f"results_{MODE}_{FEATURE_MODE}", MODEL_NAME)
EVALUATION_FILE_PATH = os.path.join(MODEL_PATH, "evaluation_on_raw_audio.csv")
CM_PLOT_PATH = os.path.join(MODEL_PATH, "confusion_matrix_on_raw_audio_poster.png") # Changed filename

model_path = BEST_MODEL_PATH
folder_a_path = "healthy_control"
folder_b_path = "parkinson_patient"

# --- 3. Load the Model ---
try:
    model = keras.models.load_model(model_path)
    print("Model loaded successfully! ✅")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# --- 4. Standardized Evaluation and Plotting Functions ---
def save_metrics_to_csv(y_true, y_pred_proba, filename, threshold=0.5):
    # (This function remains the same as your provided version)
    y_true = np.array(y_true)
    y_pred_binary = (np.array(y_pred_proba).flatten() > threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred_binary)
    if cm.shape == (1,1):
        tn, fp, fn, tp = (cm[0][0], 0, 0, 0) if y_true[0] == 0 else (0, 0, 0, cm[0][0])
    else:
        tn, fp, fn, tp = cm.ravel()
    total_samples = cm.sum()
    tn_percent = (tn / total_samples) * 100 if total_samples > 0 else 0
    fp_percent = (fp / total_samples) * 100 if total_samples > 0 else 0
    fn_percent = (fn / total_samples) * 100 if total_samples > 0 else 0
    tp_percent = (tp / total_samples) * 100 if total_samples > 0 else 0
    precision = precision_score(y_true, y_pred_binary, zero_division=0)
    recall = recall_score(y_true, y_pred_binary, zero_division=0)
    f1 = f1_score(y_true, y_pred_binary, zero_division=0)
    report_data = {'Metric': ['True Positive (TP)', 'True Negative (TN)', 'False Positive (FP)', 'False Negative (FN)', 'Precision', 'Recall (Sensitivity)', 'F1-Score'],'Value': [f"{tp} ({tp_percent:.2f}%)", f"{tn} ({tn_percent:.2f}%)", f"{fp} ({fp_percent:.2f}%)", f"{fn} ({fn_percent:.2f}%)", f"{precision:.4f}", f"{recall:.4f}", f"{f1:.4f}"]}
    pd.DataFrame(report_data).to_csv(filename, index=False)
    print(f"Evaluation results saved to: {filename}")

# <<< NEW FUNCTION FOR POSTER-QUALITY PLOT >>>
def plot_enhanced_confusion_matrix(y_true, y_pred_proba, save_path, threshold=0.5):
    """Creates a poster-quality confusion matrix with counts, percentages, and metrics."""
    y_pred_binary = (np.array(y_pred_proba) > threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    labels = np.asarray([
        [f"{tn}\n({cm_percent[0,0]:.2%})", f"{fp}\n({cm_percent[0,1]:.2%})"],
        [f"{fn}\n({cm_percent[1,0]:.2%})", f"{tp}\n({cm_percent[1,1]:.2%})"]
    ])

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=labels, fmt='', cmap='Blues', cbar=False,
                xticklabels=['Predicted Healthy', 'Predicted Parkinson'],
                yticklabels=['Actual Healthy', 'Actual Parkinson'],
                annot_kws={"size": 22})

    plt.title('Confusion Matrix on Raw Audio', fontsize=20, fontweight='bold')
    plt.xlabel(f'Specificity: {specificity:.4f}', fontsize=16)
    plt.ylabel(f'Sensitivity (Recall): {sensitivity:.4f}', fontsize=16)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Poster-quality confusion matrix saved to: {save_path}")

# --- 5. Corrected Processing and Prediction Functions ---
def process_and_predict(file_path, model):
    # (This function remains the same as the corrected version)
    try:
        audio, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION_S)
        audio = librosa.util.fix_length(audio, size=AUDIO_SAMPLES)
        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS)
        mel_spectrogram = librosa.util.fix_length(mel_spectrogram, size=EXPECTED_FRAMES, axis=1)
        y_preemp = librosa.effects.preemphasis(audio)
        mfccs = librosa.feature.mfcc(y=y_preemp, sr=sr, n_mfcc=N_MFCC, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH)
        mfccs = librosa.util.fix_length(mfccs, size=EXPECTED_FRAMES, axis=1)
        input_data = np.vstack((mel_spectrogram, mfccs))
        input_data = np.expand_dims(input_data, axis=0)
        prediction = model.predict(input_data, verbose=0)
        return prediction[0][0]
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def test_files_in_folder(folder_path, model, true_label):
    # (This function remains the same as the corrected version)
    if not os.path.exists(folder_path):
        return [], []
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    print(f"Found {len(file_list)} WAV files in {folder_path}. Processing...")
    true_labels, predicted_probs = [], []
    for file_name in file_list:
        predicted_prob = process_and_predict(os.path.join(folder_path, file_name), model)
        if predicted_prob is not None:
            true_labels.append(true_label)
            predicted_probs.append(predicted_prob)
    return true_labels, predicted_probs

# --- 6. Corrected Main Evaluation Logic ---
print("\n--- Starting Model Evaluation on Raw Audio ---")
y_true, y_pred_probs = [], []
print("\nProcessing 'healthy_control' folder (Label 0)...")
true_a, pred_a = test_files_in_folder(folder_a_path, model, true_label=0)
y_true.extend(true_a)
y_pred_probs.extend(pred_a)
print("\nProcessing 'parkinson_patient' folder (Label 1)...")
true_b, pred_b = test_files_in_folder(folder_b_path, model, true_label=1)
y_true.extend(true_b)
y_pred_probs.extend(pred_b)
print("\nEvaluation complete. ✅")

# --- 7. Generate and Display Results ---
if len(y_true) > 0 and len(y_pred_probs) > 0:
    # Save the detailed evaluation CSV
    save_metrics_to_csv(y_true, y_pred_probs, EVALUATION_FILE_PATH)

    # Generate and save the poster-quality confusion matrix plot
    plot_enhanced_confusion_matrix(y_true, y_pred_probs, CM_PLOT_PATH)
else:
    print("Not enough data to generate results. Please check file paths.")